**Türkçe için sentiment(duygu) analiz kodu**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('./drive/My Drive/')

In [0]:
# veriyi pandas ile okuyoruz
data = pd.read_csv("sentiment_data.csv")

In [5]:
df = data.copy()
df.head()

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...


In [35]:
#0->negatif veri etiketi
#1->pozitif veri etiketi
df['Rating'].unique().tolist()

[1, 0]

In [6]:
#model için gerekli kütüphaneleri import ediyoruz
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,  Dropout
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
# bütün verileri ve etiketleri listeye çeviriyoruz
target = df['Rating'].values.tolist()#negatif=0, pozitif=1
data = df['Review'].values.tolist()#text verisi

In [0]:
#veriyi test ve train verisi olarak ayırıyoruz
seperation = int(len(data) * 0.80)
x_train, x_test = data[:seperation], data[seperation:]
y_train, y_test = target[:seperation], target[seperation:]

In [9]:
#veri satır ve sütun sayısı
df.shape

(243497, 2)

In [0]:
# Verisetimizde en sık geçen 10000 kelimeyi alıyoruz
num_words = 10000

# Keras ile tokenizer tanımlıyoruz
tokenizer = Tokenizer(num_words=num_words)

In [0]:
# Veriyi tokenlara ayırıyoruz
tokenizer.fit_on_texts(data)

In [0]:
# Tokenizerı kaydediyoruz
import pickle

with open('turkish_tokenizer_hack.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# Tokenizerı yüklüyoruz
with open('turkish_tokenizer_hack.pickle', 'rb') as handle:
    turkish_tokenizer = pickle.load(handle)

In [0]:
# Train verisi olarak ayırdığımız veriyi tokenizer ile tokenize ediyoruz
x_train_tokens = turkish_tokenizer.texts_to_sequences(x_train)

In [15]:
x_train[100]

'Bu fiyata bu kalite kaçırmayın derim '

In [16]:

x_train_tokens[100]

[5, 39, 5, 131, 323, 143]

In [0]:
# Test verisi olarak ayırdığımız veriyi tokenizer ile tokenize ediyoruz

x_test_tokens = turkish_tokenizer.texts_to_sequences(x_test)


In [18]:

#Text verileri için padding yapıyoruz
#RNN ağlarını kullanırken önceden belirdiğimiz sabit bir size olur. Tüm input textlerinin sizelarını bu sabit size için padding yaparak 0 lar
#ile doldururuz.



num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
num_tokens.shape

(243497,)

In [22]:
# Bütün text verileri içinde maximum token sayısına sahip olanı buluyoruz
max_tokens = np.mean(num_tokens) + 2*np.std(num_tokens) 
max_tokens = int(max_tokens)
max_tokens

59

In [0]:
# Bütün verilere padding yapıyoruz ve bütün veriler aynı boyutta oluyor
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [25]:
# size
print(x_train_pad.shape)
print(x_test_pad.shape)

(194797, 59)
(48700, 59)


In [28]:
model = Sequential() # Kullanacağımız Keras modelini tanımlıyoruz


In [0]:
embedding_size = 50 # Her kelime için vektör boyutunu 50 olarak belirledik

In [30]:

#Kerasta bir embedding layer oluşturuyoruz ve rastgele vektörler oluşturuyoruz


# Modele embedding layer ekliyoruz
# embedding matris size = num_words * embedding_size -> 10.000 * 50
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

In [31]:
# 3-katmanlı(layer) LSTM
model.add(LSTM(units=16, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=8, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=4, return_sequences=False))
model.add(Dropout(0.2))
# Dense layer: Tek nörondan oluşuyor
model.add(Dense(1, activation='sigmoid'))# Sigmoid aktivasyon fonksiyonu


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [32]:
# Adam optimizer
from tensorflow.python.keras.optimizers import Adam
optimizer = Adam(lr=1e-3)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [33]:
# Farklı optimizerları deniyoruz
model.compile(loss='binary_crossentropy',
optimizer='adam',
metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [34]:
# modelin özeti
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 59, 50)            500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 59, 16)            4288      
_________________________________________________________________
dropout_1 (Dropout)          (None, 59, 16)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 59, 8)             800       
_________________________________________________________________
dropout_2 (Dropout)          (None, 59, 8)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dropout_3 (Dropout)          (None, 4)                

In [36]:
# epoch -> veri ile kaç kere eğiteceğiz
# batch_size -> feeding size-her epochta kaç veri ile besleyeceğiz
model.fit(x_train_pad, y_train, epochs=10, batch_size=256)




Epoch 1/10





194797/194797 [==============================] - 266s 1ms/step - loss: 0.2672 - acc: 0.9424
Epoch 2/10
194797/194797 [==============================] - 246s 1ms/step - loss: 0.1681 - acc: 0.9515
Epoch 3/10
194797/194797 [==============================] - 242s 1ms/step - loss: 0.1022 - acc: 0.9683
Epoch 4/10
194797/194797 [==============================] - 247s 1ms/step - loss: 0.0803 - acc: 0.9760
Epoch 5/10
194797/194797 [==============================] - 257s 1ms/step - loss: 0.0647 - acc: 0.9812
Epoch 6/10
194797/194797 [==============================] - 249s 1ms/step - loss: 0.0524 - acc: 0.9849
Epoch 7/10
194797/194797 [==============================] - 251s 1ms/step - loss: 0.0444 - acc: 0.9877
Epoch 8/10
194797/194797 [==============================] - 260s 1ms/step - loss: 0.0371 - acc: 0.9901
Epoch 9/10
194797/194797 [==============================] - 249s 1ms/step - loss: 0.0333 - acc: 0.9917
Epoch 10/10
194797/194797 [==============================] - 249s

In [37]:
# model sonuçları
result = model.evaluate(x_test_pad, y_test)
result

48700/48700 [==============================] - 171s 4ms/step


[0.24414843802609973, 0.9457905544147844]

In [38]:
# doğruluk oranı
accuracy = (result[1]) * 100
accuracy

94.57905544147845

In [0]:
#test yorumları(inputlar)
text1 = "böyle bir şeyi kabul edemem"
text2 = "tasarımı güzel ancak ürün açılmış tavsiye etmem"
text3 = "bu işten çok sıkıldım artık"
text4 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text5 = "yaptığın işleri hiç beğenmiyorum"
text6 = "tam bir fiyat performans ürünü beğendim"
text7 = "Bu ürünü beğenmedim"
texts = [text1, text2,text3,text4,text5,text6,text7]

In [0]:
tokens = turkish_tokenizer.texts_to_sequences(texts)

In [41]:
tokens = turkish_tokenizer.texts_to_sequences(texts)
tokens

[[104, 2, 1032, 2333, 1466, 5, 4, 5, 1779],
 [553, 61, 82, 27, 1, 458, 33, 3, 4, 9, 1031],
 [46, 1096, 6419, 20, 50],
 [177, 735, 7728, 82, 263, 105, 326, 16],
 [46, 5, 30, 177, 2, 1717, 1244, 19, 677, 83],
 [74, 2, 28, 111, 19],
 [7, 2, 4, 50]]

In [0]:
#padding
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)

In [62]:
#model bu yorumların hangi duyguya yakın olduğunu tahminliyor
model.predict(tokens_pad)

array([[0.00613517],
       [0.08107951],
       [0.00456542],
       [0.9993015 ],
       [0.0052653 ],
       [0.99948347],
       [0.96659845]], dtype=float32)

In [61]:

for i in model.predict(tokens_pad):
    if i < 0.5:
        print("negatif")#negatif yorum yapmış
    else
        print("pozitif")#pozitif yorum yapmış


negative
negative
negative
positive
negative
positive
positive


In [0]:
from keras.models import load_model

model.save('hack_model.h5')  # modeli kaydediyoruz
